In [1]:
import numpy as np;
import pandas as pd;

In [14]:
## Parameters and Global variables

ROOT_DIR = "/Users/sreeramh/Documents/Kerala_Covid/SpatioTemporal/gitsrc/SpatioTemporalSEIR"

cw = 0.2

In [11]:
initDataDF = pd.read_csv(ROOT_DIR + "/data/sample_init_data.csv")
r = len(initDataDF.index)


tData = pd.read_csv(ROOT_DIR + "/output/tmatrix_results.csv")

In [9]:
tData.head(5)
# TODO fix the unnamed colum

,Unnamed: 0,iName,jName,Tij
0,0,P1,P1,29700.000000
1,1,P1,P1.1,14.285714
2,2,P1,P1.2,14.285714
3,3,P1,P1.1.1,16.666667
4,4,P1,P1.1.2,16.666667


In [13]:
tMatrix = tData['Tij'].to_numpy().reshape(r-1,r-1)
tMatrix

array([[2.97000000e+04, 1.42857143e+01, 1.42857143e+01, 1.66666667e+01,
        1.66666667e+01, 1.66666667e+01],
       [6.25000000e+00, 1.48500000e+04, 3.57142857e+00, 1.92307692e+00,
        1.92307692e+00, 1.50000000e+01],
       [6.25000000e+00, 3.57142857e+00, 1.48500000e+04, 1.50000000e+01,
        1.50000000e+01, 1.92307692e+00],
       [2.14285714e+00, 8.92857143e-01, 1.25000000e+00, 7.42500000e+03,
        6.81818182e-01, 6.25000000e+00],
       [2.14285714e+00, 8.92857143e-01, 1.25000000e+00, 6.81818182e-01,
        7.42500000e+03, 6.25000000e+00],
       [2.14285714e+00, 1.25000000e+00, 8.92857143e-01, 6.25000000e+00,
        6.25000000e+00, 7.42500000e+03]])

In [8]:
initDataDF.head(5)

,name,S,E,I,H,R
0,P1,30000,0,1,0,0
1,P1.1,15000,0,0,0,0
2,P1.2,15000,0,0,0,0
3,P1.1.1,7500,0,0,0,0
4,P1.1.2,7500,0,0,0,0


In [24]:
def getCij(i, j, tMatrix, initDataDF):
    Ni = initDataDF.iloc[i]['S']
    cwSum = 0
    #print("Ni = ", Ni)
    for k in range(0,r-1):
        Tik = tMatrix[i][k]
        #print("Tik = ", Tik)

        Tjk = tMatrix[j][k]
        #print("Tjk = ", Tjk)

        TlkSum = 0;
        for l in range(0, r-1):
            TlkSum = TlkSum + tMatrix[l][k]
        
        
        #print("TlkSum = ", TlkSum)
        cwSum = cwSum + (Tik/Ni)*(Tjk/TlkSum)
    return cw*cwSum

In [25]:
getCij(0,1, tMatrix, initDataDF)

0.0001370476162494559

In [26]:
resultsDF = pd.DataFrame(columns=['iName', 'jName', 'Cij'], );
for i in range(0,r-1):
    for j in range(0,r-1):
        Tij = getCij(i,j,tMatrix, initDataDF)
        resultsDF = resultsDF.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Cij': Tij},ignore_index=True)
 

In [27]:
resultsDF.to_csv(ROOT_DIR + "/output/cmatrix_results.csv")

In [28]:
resultsDF.head(10)

,iName,jName,Cij
0,P1,P1,0.197875
1,P1,P1.1,0.000137
2,P1,P1.2,0.000137
3,P1,P1.1.1,0.000125
4,P1,P1.1.2,0.000125
5,P1,P1.2.1,0.000125
6,P1.1,P1,0.000274
7,P1.1,P1.1,0.197722
8,P1.1,P1.2,0.000095
9,P1.1,P1.1.1,0.000038
